In [1]:
import tensorflow as tf
import numpy as np
import os

## Loading CIFAR10 Data

In [2]:
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
# Reducing Target innecessary dimension
y_test  = np.squeeze(y_test)

## Creating Dataset Objects

In [3]:
# Normalize Images
def normalize_img(x_, y_):
    return tf.cast(x_, tf.float32) / 255., y_

# 1-hot encoding
def to_categorical(x_, y_):
    return x_, tf.one_hot(y_, depth=10)

# Resizing 
def process_images(image, label):
    # Resize images from 32x32 to 224x224
    image = tf.image.resize(image, (224,224))
    return image, label

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.map(normalize_img)
test_dataset = test_dataset.map(to_categorical)
test_dataset = test_dataset.map(process_images)
test_dataset = test_dataset.batch(1)

## Quantization Layer

In [4]:
def Quantization_layer(tensor, Quantization = True,signed = True, word_size = 12, frac_size = 6):
    
    factor = 2.0**frac_size
    
    # Quantized max and min values, in case of the need to implement overflow cases.
    #if signed:
    #    Max_Qvalue = ((1 << (word_size-1)) - 1)/factor
    #    Min_Qvalue = -Max_Qvalue - 1
    #else:
    #    Max_Qvalue = ((1 << (word_size)) - 1)/factor
    #    Min_Qvalue = 0
    
    if Quantization:
        return tf.round(tensor*factor) / factor             #Quantization, assuming no overflow
    else:
        return tensor                                       #Simple Bypass

## Creating VGG Model

In [5]:
from tensorflow.keras.layers import Conv2D, Dense, MaxPool2D, Flatten, Lambda


def build_model(input_layer, Quantization = True, signed = True, word_size = 12, frac_size = 6 ):
    
    Arguments = {'Quantization':Quantization, 'signed':signed, 'word_size':word_size, 'frac_size':frac_size}
    QInp      = Lambda(Quantization_layer, arguments = Arguments )(input_layer)
    
    #Conv Block
    Conv1   = Conv2D(filters=64,kernel_size=(3,3),padding="same")(QInp)
    QConv1  = Lambda(Quantization_layer, arguments = Arguments )(Conv1)
    Relu1   = tf.keras.activations.relu(QConv1)
    QRelu1  = Lambda(Quantization_layer, arguments = Arguments )(Relu1)
    
    Conv2   = Conv2D(filters=64,kernel_size=(3,3),padding="same")(QRelu1)
    QConv2  = Lambda(Quantization_layer, arguments = Arguments )(Conv2)
    Relu2   = tf.keras.activations.relu(QConv2)
    QRelu2  = Lambda(Quantization_layer, arguments = Arguments )(Relu2)
    MP2     = MaxPool2D(pool_size=(2,2),strides=(2,2))(QRelu2)
    
    Conv3   = Conv2D(filters=128, kernel_size=(3,3), padding="same")(MP2)
    QConv3  = Lambda(Quantization_layer, arguments = Arguments )(Conv3)
    Relu3   = tf.keras.activations.relu(QConv3)
    QRelu3  = Lambda(Quantization_layer, arguments = Arguments )(Relu3)
    
    Conv4   = Conv2D(filters=128, kernel_size=(3,3), padding="same")(QRelu3)
    QConv4  = Lambda(Quantization_layer, arguments = Arguments )(Conv4)
    Relu4   = tf.keras.activations.relu(QConv4)
    QRelu4  = Lambda(Quantization_layer, arguments = Arguments )(Relu4)
    MP4     = MaxPool2D(pool_size=(2,2),strides=(2,2))(QRelu4)
    
    Conv5   = Conv2D(filters=256, kernel_size=(3,3), padding="same")(MP4)
    QConv5  = Lambda(Quantization_layer, arguments = Arguments )(Conv5)
    Relu5   = tf.keras.activations.relu(QConv5)
    QRelu5  = Lambda(Quantization_layer, arguments = Arguments )(Relu5)
    
    Conv6   = Conv2D(filters=256, kernel_size=(3,3), padding="same")(QRelu5)
    QConv6  = Lambda(Quantization_layer, arguments = Arguments )(Conv6)
    Relu6   = tf.keras.activations.relu(QConv6)
    QRelu6  = Lambda(Quantization_layer, arguments = Arguments )(Relu6)
    
    Conv7   = Conv2D(filters=256, kernel_size=(3,3), padding="same")(QRelu6)
    QConv7  = Lambda(Quantization_layer, arguments = Arguments )(Conv7)
    Relu7   = tf.keras.activations.relu(QConv7)
    QRelu7  = Lambda(Quantization_layer, arguments = Arguments )(Relu7)
    MP7     = MaxPool2D(pool_size=(2,2),strides=(2,2))(QRelu7)
    
    Conv8   = Conv2D(filters=512, kernel_size=(3,3), padding="same")(MP7)
    QConv8  = Lambda(Quantization_layer, arguments = Arguments )(Conv8)
    Relu8   = tf.keras.activations.relu(QConv8)
    QRelu8  = Lambda(Quantization_layer, arguments = Arguments )(Relu8)
    
    Conv9   = Conv2D(filters=512, kernel_size=(3,3), padding="same")(QRelu8)
    QConv9  = Lambda(Quantization_layer, arguments = Arguments )(Conv9)
    Relu9   = tf.keras.activations.relu(QConv9)
    QRelu9  = Lambda(Quantization_layer, arguments = Arguments )(Relu9)
    
    Conv10   = Conv2D(filters=512, kernel_size=(3,3), padding="same")(QRelu9)
    QConv10  = Lambda(Quantization_layer, arguments = Arguments )(Conv10)
    Relu10   = tf.keras.activations.relu(QConv10)
    QRelu10  = Lambda(Quantization_layer, arguments = Arguments )(Relu10)
    MP10     = MaxPool2D(pool_size=(2,2),strides=(2,2))(QRelu10)
    
    Conv11   = Conv2D(filters=512, kernel_size=(3,3), padding="same")(MP10)
    QConv11  = Lambda(Quantization_layer, arguments = Arguments )(Conv11)
    Relu11   = tf.keras.activations.relu(QConv11)
    QRelu11  = Lambda(Quantization_layer, arguments = Arguments )(Relu11)
    
    Conv12   = Conv2D(filters=512, kernel_size=(3,3), padding="same")(QRelu11)
    QConv12  = Lambda(Quantization_layer, arguments = Arguments )(Conv12)
    Relu12   = tf.keras.activations.relu(QConv12)
    QRelu12  = Lambda(Quantization_layer, arguments = Arguments )(Relu12)
    
    Conv13   = Conv2D(filters=512, kernel_size=(3,3), padding="same")(QRelu12)
    QConv13  = Lambda(Quantization_layer, arguments = Arguments )(Conv13)
    Relu13   = tf.keras.activations.relu(QConv13)
    QRelu13  = Lambda(Quantization_layer, arguments = Arguments )(Relu13)
    MP13     = MaxPool2D(pool_size=(2,2),strides=(2,2))(QRelu13)
    
    Flat    = Flatten()(MP13)
    
    Dense14  = Dense(4096)(Flat)
    QDense14 = Lambda(Quantization_layer, arguments = Arguments )(Dense14)
    Relu14   = tf.keras.activations.relu(QDense14)
    QRelu14  = Lambda(Quantization_layer, arguments = Arguments )(Relu14)
    
    Dense15  = Dense(4096)(QRelu14)
    QDense15 = Lambda(Quantization_layer, arguments = Arguments )(Dense15)
    Relu15   = tf.keras.activations.relu(QDense15)
    QRelu15  = Lambda(Quantization_layer, arguments = Arguments )(Relu15)
    
    Dense16  = Dense(10)(QRelu15)
    QDense16 = Lambda(Quantization_layer, arguments = Arguments )(Dense16)
    SM16     = tf.keras.activations.softmax(QDense16)
    QSM16    = Lambda(Quantization_layer, arguments = Arguments )(SM16)
    
    return QSM16

## Loading Quantized model and Non Quantized model 

En base al resultado del ultimo item del cuaderno "VGG Analysis" las entradas/salidas de las capas tienen valores que varian entre -27 y 44.03, Salvo la ultima capa densa, las activaciones de todas las demas capas estan contenidas en valores bajos (magnitud menor a 5)

Considerando un bit para el signo, se necesitan al menos 6 bits para la parte entera, se medirá el accuracy de modelos en funcion del numero de bits destinados a la parte fraccionaria.

In [11]:
input_layer   = tf.keras.Input((224,224,3))
output_layer  = build_model(input_layer, Quantization = False)

Frac_Bits = 6
Int_Bits  = 6

Qinput_layer  = tf.keras.Input((224,224,3))
Qoutput_layer = build_model(Qinput_layer, Quantization = True, word_size = (Frac_Bits+Int_Bits+1), frac_size = Frac_Bits)

VGG16  = tf.keras.Model(inputs=input_layer, outputs=output_layer)
QVGG16 = tf.keras.Model(inputs=Qinput_layer, outputs=Qoutput_layer)

## Loading Pretrained Weights

In [12]:
# Loading Wieghts
cwd = os.getcwd()
Wgt_dir = os.path.join(cwd,'TrainedWeights')
Wgt_dir = os.path.join(Wgt_dir,'Weights')
VGG16.load_weights(Wgt_dir)
QVGG16.load_weights(Wgt_dir)

# Visualize AlexNet Architecture
#VGG16.summary()

# Visualize initialized weights
#VGG16.weights

## Quantize weights

En base al resultado del ultimo item del cuaderno "VGG Analysis" los pesos valores que varian entre -0.1 y 0.1.

Considerando un bit para el signo, se necesitan al menos 1 bits para la parte entera, se medirá el accuracy de modelos en funcion del numero de bits destinados a la parte fraccionaria.

In [13]:
def Quantization(List, Quantization = True, signed = True, word_size = 12, frac_size = 6):
    factor = 2.0**frac_size
    return tf.round(np.array(List)*factor) / factor             #Quantization, assuming no overflow

Frac_Bits = 6
Int_Bits  = 1

for layer in QVGG16.layers:
    weights = layer.get_weights()
    if weights:                     # Layer with weights
        # Quantization of Weights and Bias 
        Qweights    = [Quantization(itm, word_size = (Frac_Bits+Int_Bits+1), frac_size = Frac_Bits) for itm in weights]
        layer.set_weights(Qweights)

## Check the general Accuracy

In [14]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
VGG16.compile(optimizer=optimizer, loss=loss, metrics=metrics)
QVGG16.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## modelo original

In [10]:
VGG16.evaluate(test_dataset)

10000/10000 [==============================] - 141s 14ms/step - loss: 0.8012 - accuracy: 0.7349


[0.8012385272836876, 0.7349]

## 4 bits de precision para pesos && 4 bits de precision para activaciones

In [17]:
QVGG16.evaluate(test_dataset)

10000/10000 [==============================] - 176s 18ms/step - loss: 2.3026 - accuracy: 0.1000


[2.3025851249694824, 0.1]

## 5 bits de precision para pesos && 5 bits de precision para activaciones

In [10]:
QVGG16.evaluate(test_dataset)

10000/10000 [==============================] - 173s 17ms/step - loss: 1.6283 - accuracy: 0.6467


[1.6282763871062547, 0.6467]

## 6 bits de precision para pesos && 6 bits de precision para activaciones

In [15]:
QVGG16.evaluate(test_dataset)

10000/10000 [==============================] - 176s 18ms/step - loss: 1.0620 - accuracy: 0.7302


[1.061964491548948, 0.7302]

5 bits son necesarios para un buen resultado, con 6 se alcanza los resultados del modelo original, con 4 bits o menos es practicamente un clasificador aleatorio